# Regex

In [35]:
import pandas as pd 
import numpy as np
import re

In [44]:
data = pd.read_csv('./dataset/data_web.csv')
df = data.copy()

# renomeando as cols
df.rename( columns= lambda x: x.lower(), inplace=True)


In [37]:
# transfommacao
# product_name
df['product_name'] = df['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price - Remover o símbolo '$' e converter para float
df['product_price'] = df['product_price'].apply(lambda x: x.replace('$', '') if isinstance(x, str) else x ).astype(float)

# scrapy_datetime
df['scrapy_datetime'] = pd.to_datetime(df['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S:')

# Color Name
df['color_name'] = df['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if isinstance(x, str) else x)

# Fit
df['fit'] = df['fit'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

# Composition
# df['composition'] = df['composition'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

# Size
df['size'] = df['size'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

# size_number - Extraido números da coluna 'size'
df['size_number'] = df['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if isinstance(x, str) and re.search('\d{3}cm', x) else x)
df['size_number'] = df['size_number'].apply( lambda x: re.search('\d+', x).group(0) if pd.notnull( x ) else x )

# size_model - Extraido números da coluna 'size'
df['size_model'] = df['size'].apply( lambda x: re.search('\d+/\\d+', x).group(0) if isinstance(x, str) and re.search('\d+/\\d+', x) else x  )
# jeito mai simples df['size_model'] = df['size'].str.extract( '(\d+/\\d+)' )

# brack composition by comma
df1 = df['composition'].str.split(',', expand=True)

In [42]:
# criando um dataframe vazio do tamanho de data para alocar as colunas em ordem
df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton', 'Spandex', 'Elastomultiester'] )

# DF Cotton
df_cotton = df1[0] # data que tem só cotton
df_cotton.name = 'cotton' # passo o name para a coluna

df_ref = pd.concat([ df_ref, df_cotton], axis=1) # faco a uniao com concat 
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')] # e dorp a col original q era so NaN

# DF Spandex
df_spandex = df1.loc[df1[1].str.contains('Spandex', na=False), 1]
df_spandex.name = 'spandex'

df_ref = pd.concat([df_ref, df_spandex], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# DF Elastomultiester	
df_Elastomultiester	 = df1.loc[ df1[1].str.contains('Elastomultiester', na=False), 1]
df_Elastomultiester.name = 'Elastomultiester'

df_ref = pd.concat([ df_ref, df_Elastomultiester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

df_ref = df_ref.drop('Spandex', axis=1)

# final join
data = pd.concat( [ df, df_ref ], axis=1 )

# format composition data
# cotton
data['cotton'] = data['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )

#spandex.
	
data['spandex'] = data['spandex'].apply( lambda x: int( re.search('\d+', x).group(0) ) /100 if pd.notnull(x) else x )

# Elastomultiester

data['Elastomultiester'] = data['Elastomultiester'].apply( lambda x: int(re.search('\d+', x).group(0))/ 100 if pd.notnull( x ) else x)


In [43]:
data.head()

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,color_id,color_name,fit,composition,pocket lining,size,size_number,size_model,cotton,spandex,Elastomultiester
0,1130309005,baggy_jeans,men_jeans_loose,39.99,2023-10-04 14:36:16,1130309,5,denim_black,loose_fit,Cotton 100%,"Polyester 65%, Cotton 35%","the_model_is_187cm/6'2""_and_wears_a_size_31/32",187,31/32,1.0,NaN,NaN
1,1130309005,baggy_jeans,men_jeans_loose,39.99,2023-10-04 14:36:16,1130309,5,light_denim_blue,loose_fit,Cotton 100%,"Polyester 65%, Cotton 35%","the_model_is_187cm/6'2""_and_wears_a_size_31/32",187,31/32,1.0,NaN,NaN
2,1130309005,baggy_jeans,men_jeans_loose,39.99,2023-10-04 14:36:16,1130309,5,dark_denim_gray,loose_fit,Cotton 100%,"Polyester 65%, Cotton 35%","the_model_is_187cm/6'2""_and_wears_a_size_31/32",187,31/32,1.0,NaN,NaN
3,1130309005,baggy_jeans,men_jeans_loose,39.99,2023-10-04 14:36:16,1130309,5,dark_denim_gray,loose_fit,Cotton 100%,"Polyester 65%, Cotton 35%","the_model_is_187cm/6'2""_and_wears_a_size_31/32",187,31/32,1.0,NaN,NaN
4,1130309005,baggy_jeans,men_jeans_loose,39.99,2023-10-04 14:36:16,1130309,5,dark_denim_blue,loose_fit,Cotton 100%,"Polyester 65%, Cotton 35%","the_model_is_187cm/6'2""_and_wears_a_size_31/32",187,31/32,1.0,NaN,NaN


In [ ]:

# Cotton
df_cottom = df1[0]
df_cottom.name = 'cotton' # renemeei a col com name

df_ref = pd.concat([ df_ref, df1[ 0 ] ], axis=1 ) # juntei a minha col 0 no new dataframe
df_ref = df_ref.iloc[ :, ~df_ref.columns.duplicated( keep='last' ) ] # tirar as colunas duplicadas do data frame\ keep lasta mantem a col que precisamos

# Spandex 
df_Spandex = df1.loc[ df1[ 1 ].str.contains( 'Spandex', na=True ), 1 ]
df_Spandex.name = 'Spandex'

df_ref = pd.concat( [ df_ref, df_Spandex ], axis=1 )
df_Spandex = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]


# Elastomultiester 
df_Elastomultiester = df1.loc[df1[ 1 ].str.contains( 'Elastomultiester', na=True), 1 ] # selecionei so as ''
df_Elastomultiester.name = 'Elastomultiester'

df_ref = pd.concat([ df_ref, df_Spandex ], axis=1)# agr concat para ir para meu dataframe 
df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]

df_ref.sample(4)